In [7]:
import time
import numpy as np
from memory_profiler import memory_usage

np.random.seed(42)

def get_minor(matrix, i, j):
    return [row[:j] + row[j+1:] for k, row in enumerate(matrix) if k != i]

def determinant(matrix):
    n = len(matrix)
    if n == 1:
        return matrix[0][0]
    if n == 2:
        return matrix[0][0]*matrix[1][1] - matrix[0][1]*matrix[1][0]
    return sum((-1)**j * matrix[0][j] * determinant(get_minor(matrix, 0, j)) for j in range(n))

def cofactor_matrix(matrix):
    n = len(matrix)
    return [[(-1)**(i + j) * determinant(get_minor(matrix, i, j)) for j in range(n)] for i in range(n)]

def transpose(matrix):
    return [list(row) for row in zip(*matrix)]

def adjugate_matrix(matrix):
    return transpose(cofactor_matrix(matrix))

def invert_matrix(matrix):
    det = determinant(matrix)
    if det == 0:
        raise ValueError("Zero determinant")
    adj = adjugate_matrix(matrix)
    return [[elem / det for elem in row] for row in adj]

def generate_random_matrix(n):
    return np.random.uniform(1, 10, (n, n)).tolist()

def compute_error(A, A_inv):
    A_np = np.array(A)
    A_inv_np = np.array(A_inv)
    identity = np.eye(len(A_np))
    return np.linalg.norm(A_np @ A_inv_np - identity)

def wrapper(func, *args):
    return func(*args)

if __name__ == '__main__':
    matrix_sizes = [3, 5, 10]
    for size in matrix_sizes:
        A = generate_random_matrix(size)
        try:
            cond_number = np.linalg.cond(A)
            start_time = time.time()
            mem_usage, A_inv = memory_usage((wrapper, (invert_matrix, A)), retval=True, max_iterations=1)
            end_time = time.time()
            execution_time = end_time - start_time
            peak_memory = max(mem_usage) - min(mem_usage)
            error = compute_error(A, A_inv)
            print(f"\nSize {size}x{size}")
            print(f"Condition number: {cond_number:.2e}")
            print(f"Execution time: {execution_time:.6f} seconds")
            print(f"Peak memory usage: {peak_memory:.6f} MiB")
            print(f"Error ||AA⁻¹ - I||: {error:.2e}")
            print(f"Inverse matrix {size}x{size}:")
            for row in A_inv:
                print(row)
        except Exception as e:
            print(f"\nSize {size}x{size}: FAILED — {e}")



Size 3x3
Condition number: 9.79e+01
Execution time: 0.808432 seconds
Peak memory usage: 23.062500 MiB
Error ||AA⁻¹ - I||: 8.93e-15
Inverse matrix 3x3:
[-0.3338957729658942, 0.3193378369678932, 0.27549155029914535]
[-2.1714530401770302, 0.9587517291078481, 2.210920090998607]
[3.0588976142457387, -1.3914206352912173, -2.943172918220214]

Size 5x5
Condition number: 5.98e+01
Execution time: 0.619284 seconds
Peak memory usage: 2.546875 MiB
Error ||AA⁻¹ - I||: 9.12e-15
Inverse matrix 5x5:
[0.169781419305884, -0.5682211377317494, -1.1965536034224162, 1.0749411984981618, 0.06484301700544656]
[-0.07245064767332397, 0.1230955003902323, 0.8784443883090669, -0.5649359034078915, -0.06169841183376517]
[0.07469408253111619, 0.10180391271390457, 0.8679204724513349, -0.7929014392346987, 0.060351998022869786]
[-0.11292593103445728, 0.3906923741122465, 0.15280465373405033, -0.10044649322706418, -0.1678640903835781]
[0.022804716193905686, -0.09097247726604835, -0.6737039752717503, 0.45059490711735956, 0.